In [1]:
from subprocess import check_output
print(check_output(["dir","C:\\Users\\Lenovo NBB\\Desktop\\Dissertation Shelf\\Code"],shell=True).decode("utf8"))

 Volume in drive C is Windows8_OS
 Volume Serial Number is 089C-5A24

 Directory of C:\Users\Lenovo NBB\Desktop\Dissertation Shelf\Code

2019-08-23  03:38 PM    <DIR>          .
2019-08-23  03:38 PM    <DIR>          ..
2019-08-13  03:47 PM    <DIR>          .ipynb_checkpoints
2019-08-23  03:38 PM           739,517 .RData
2019-08-23  03:38 PM            20,582 .Rhistory
2019-08-07  01:04 PM         3,182,726 2017-2019-datetime-clean.csv
2019-08-07  03:19 PM         3,261,603 2017-2019-datetime.csv
2019-07-26  10:28 PM         2,582,868 2017-2019-datetimev.csv
2019-07-28  01:48 PM         3,240,718 2017-2019-mid-datetimehour.csv
2019-07-23  01:33 PM         3,304,954 2017-2019-mid.csv
2019-07-20  02:21 AM             1,413 APIconnection.py
2019-08-13  03:47 PM           327,663 ARIMA Models.ipynb
2019-07-15  04:24 PM                15 BMRSAPIKey.txt
2019-07-24  01:07 PM            13,574 Gas Data.docx
2019-08-05  09:10 AM            54,828 GasData.csv
2019-08-12  06:06 PM            55,

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv("2017-2019-mid.csv",parse_dates=[2])
df.head()


,Record Type,Data Provider,Settlement Date,Settlement Period,Price,Volume
0,MID,APXMIDP,2017-01-01,1,44.90,505.00
1,MID,APXMIDP,2017-01-01,2,47.13,559.15
2,MID,APXMIDP,2017-01-01,3,45.62,649.35
3,MID,APXMIDP,2017-01-01,4,44.05,712.65
4,MID,APXMIDP,2017-01-01,5,44.83,599.15


In [5]:
df.shape

(87075, 6)

In [6]:
(df == 0).sum()

Record Type              0
Data Provider            0
Settlement Date          0
Settlement Period        0
Price                43560
Volume               43560
dtype: int64

In [7]:
df = df[(df != 0).all(1)]
df.shape

(43515, 6)

In [8]:
df['Record Type'].unique()

array(['MID'], dtype=object)

In [9]:
df.groupby('Data Provider').size()

Data Provider
APXMIDP     43395
N2EXMIDP      120
dtype: int64

In [10]:
df = df[df['Data Provider'] != 'N2EXMIDP']
df = df.iloc[:,2:] #Selecting data by rows and colummns [row selection, column selection]
df.head()



,Settlement Date,Settlement Period,Price,Volume
0,2017-01-01,1,44.90,505.00
1,2017-01-01,2,47.13,559.15
2,2017-01-01,3,45.62,649.35
3,2017-01-01,4,44.05,712.65
4,2017-01-01,5,44.83,599.15


In [11]:
#Adjusting time 
d = dict(zip(range(1, 49),pd.timedelta_range('00:00:00', '23:30:00', freq='30T')))
#Adding time column 
df['time'] = pd.to_timedelta(df['Settlement Period'].map(d), unit='h')
df.head()

,Settlement Date,Settlement Period,Price,Volume,time
0,2017-01-01,1,44.90,505.00,00:00:00
1,2017-01-01,2,47.13,559.15,00:30:00
2,2017-01-01,3,45.62,649.35,01:00:00
3,2017-01-01,4,44.05,712.65,01:30:00
4,2017-01-01,5,44.83,599.15,02:00:00


In [12]:
#Let's add date and time together
df['datetime'] = pd.to_datetime(df['Settlement Date'] + df['time'])
df = df.reset_index().set_index('datetime')
df.head(10)

,index,Settlement Date,Settlement Period,Price,Volume,time
datetime,,,,,,
2017-01-01 00:00:00,0,2017-01-01,1,44.90,505.00,00:00:00
2017-01-01 00:30:00,1,2017-01-01,2,47.13,559.15,00:30:00
2017-01-01 01:00:00,2,2017-01-01,3,45.62,649.35,01:00:00
2017-01-01 01:30:00,3,2017-01-01,4,44.05,712.65,01:30:00
2017-01-01 02:00:00,4,2017-01-01,5,44.83,599.15,02:00:00
2017-01-01 02:30:00,5,2017-01-01,6,45.51,499.30,02:30:00
2017-01-01 03:00:00,6,2017-01-01,7,44.85,474.50,03:00:00
2017-01-01 03:30:00,7,2017-01-01,8,42.80,487.00,03:30:00
2017-01-01 04:00:00,8,2017-01-01,9,42.33,348.85,04:00:00


In [13]:
#Using Feature Engineering from pandas index 
df = df[['Price', 'Volume', 'Settlement Period']]
df['hour'] = df.index.hour
#df['dayofweek'] = df.index.dayofweek
df['dayofweek'] = df.index.strftime('%a')
df['quarter'] = df.index.quarter
#df['month'] = df.index.month
df['month'] = df.index.strftime('%b')
df['year'] = df.index.year
df['dayofyear'] = df.index.dayofyear
df['dayofmonth'] = df.index.day
df['weekofyear'] = df.index.weekofyear

df

,Price,Volume,Settlement Period,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear
datetime,,,,,,,,,,,
2017-01-01 00:00:00,44.90,505.00,1,0.0,Sun,1.0,Jan,2017.0,1.0,1.0,52.0
2017-01-01 00:30:00,47.13,559.15,2,0.0,Sun,1.0,Jan,2017.0,1.0,1.0,52.0
2017-01-01 01:00:00,45.62,649.35,3,1.0,Sun,1.0,Jan,2017.0,1.0,1.0,52.0
2017-01-01 01:30:00,44.05,712.65,4,1.0,Sun,1.0,Jan,2017.0,1.0,1.0,52.0
2017-01-01 02:00:00,44.83,599.15,5,2.0,Sun,1.0,Jan,2017.0,1.0,1.0,52.0
2017-01-01 02:30:00,45.51,499.30,6,2.0,Sun,1.0,Jan,2017.0,1.0,1.0,52.0
2017-01-01 03:00:00,44.85,474.50,7,3.0,Sun,1.0,Jan,2017.0,1.0,1.0,52.0
2017-01-01 03:30:00,42.80,487.00,8,3.0,Sun,1.0,Jan,2017.0,1.0,1.0,52.0
2017-01-01 04:00:00,42.33,348.85,9,4.0,Sun,1.0,Jan,2017.0,1.0,1.0,52.0


In [14]:
df.dtypes

Price                float64
Volume               float64
Settlement Period      int64
hour                 float64
dayofweek             object
quarter              float64
month                 object
year                 float64
dayofyear            float64
dayofmonth           float64
weekofyear           float64
dtype: object

In [15]:
df.isna().sum()


Price                0
Volume               0
Settlement Period    0
hour                 4
dayofweek            0
quarter              4
month                0
year                 4
dayofyear            4
dayofmonth           4
weekofyear           4
dtype: int64

In [16]:
df.shape


(43395, 11)

In [17]:
df = df.dropna()

In [18]:
df.to_csv('2017-2019-datetime.csv')


In [19]:
np.mean(df['Volume'])

799.3861710953897

In [20]:
np.mean(df['Price'])

49.81772994399753